In [1]:
# import dependencies
import pandas as pd
import pyodbc
import psycopg2

# define a connection string to Access database
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\23470\OneDrive\Desktop\10Alytics\capstone_project\project3\WPI.accdb;'
    )
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# retrieve table names from the Access database
table_names = [table.table_name for table in cursor.tables(tableType='TABLE')]

# save tables to csv file on local machine
for table_name in table_names:
    df = pd.read_sql_query(f'SELECT * FROM [{table_name}]', conn)
    df.to_csv(f'{table_name.replace("/", "_")}.csv', index=False)
    
# closs connection 
cursor.close()
conn.close()

In [18]:
# define a connect to postgres sever 
def get_db_connection(host, database, username, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=username,
        password=password
    )
    
    return conn

# initialize connection function parameters
host = "localhost"
database = "World_Port_index"
username = "postgres"
password = "kw#s#k00"

# establish a connect
con = get_db_connection(host, database, username, password)
# create cursor
curr = con.cursor()

In [3]:
# convert the csv file of the Wpi Data to a dataframe for easy upload to postgres database
ndf = pd.read_csv('Wpi Data.csv')
ndf = pd.DataFrame(ndf)

In [17]:
# upload the dataframe to postgress database
# NB: a database called World_Port_index and table name wpi_data has already been created in postgres server 
with open('Wpi Data.csv', 'r') as file:
    next(file)  # Skip the header row if needed

    # Use the COPY statement to upload the data
    curr.copy_from(file, 'wpi_data', sep=',')
    con.commit()
    
# close connection   
curr.close()
con.close()